## 1. Imports

In [21]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [22]:
# configuration pour une meilleure lisibilité
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

# chargement du dataset
df = pd.read_csv('../../raw_data/top_50_in_73_countries_2023_2025.csv')
print(f"Dimensions initiales : {df.shape}")
df.head()

Dimensions initiales : (2110316, 25)


,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,duration_ms,album_name,album_release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2RkZ5LkEzeHGRsmDqKwmaJ,Ordinary,Alex Warren,1,1,0,NaN,2025-06-11,95,False,186964,"You'll Be Alright, Kid (Chapter 1)",2024-09-26,0.368,0.694,2,-6.141,1,0.0600,0.704000,0.000007,0.0550,0.391,168.115,3
1,42UBPzRMh5yyz0EDPr6fr1,Manchild,Sabrina Carpenter,2,-1,48,NaN,2025-06-11,89,True,213645,Manchild,2025-06-05,0.731,0.685,7,-5.087,1,0.0572,0.122000,0.000000,0.3170,0.811,123.010,4
2,0FTmksd2dxiE5e3rWyJXs6,back to friends,sombr,3,0,1,NaN,2025-06-11,98,False,199032,back to friends,2024-12-27,0.436,0.723,1,-2.291,1,0.0301,0.000094,0.000088,0.0929,0.235,92.855,4
3,7so0lgd0zP2Sbgs2d7a1SZ,Die With A Smile,"Lady Gaga, Bruno Mars",4,0,-1,NaN,2025-06-11,91,False,251667,MAYHEM,2025-03-07,0.519,0.601,6,-7.727,0,0.0317,0.289000,0.000000,0.1260,0.498,157.964,3
4,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,5,1,0,NaN,2025-06-11,100,False,210373,HIT ME HARD AND SOFT,2024-05-17,0.747,0.507,2,-10.171,1,0.0358,0.200000,0.060800,0.1170,0.438,104.978,4


## 2. Exploration et nettoyage des données

In [23]:
# a partir de la colonne 'snapshot_date', extraire les années uniques des dates uniques
def get_unique_years(df):
    unique_dates = df['snapshot_date'].unique()
    unique_years = sorted({date.split('-')[0] for date in unique_dates})
    return unique_years

unique_years = get_unique_years(df)
print(unique_years)

['2023', '2024', '2025']


In [24]:
# retourne le nombre de lignes pour chaque date unique dans la colonne 'snapshot_date'
def count_rows_per_snapshot_date(df):
    snapshot_date_counts = df['snapshot_date'].value_counts().sort_index()
    return snapshot_date_counts

snapshot_date_counts = count_rows_per_snapshot_date(df)
print(snapshot_date_counts)

snapshot_date
2023-10-18    3641
2023-10-19    3641
2023-10-20    3642
2023-10-21    3645
2023-10-22    3650
              ... 
2025-06-07    3600
2025-06-08    3600
2025-06-09    3600
2025-06-10    3600
2025-06-11    3600
Name: count, Length: 583, dtype: int64


In [25]:
# retourne les dates de snapshot ayant exactement target_count lignes
def get_full_snapshot_dates(df, target_count=3650):
    snapshot_date_counts = df['snapshot_date'].value_counts()
    full_snapshot_dates = snapshot_date_counts[snapshot_date_counts == target_count].index.tolist()
    return full_snapshot_dates

full_snapshot_dates = get_full_snapshot_dates(df)
print(sorted(full_snapshot_dates))

['2023-10-22', '2023-10-29', '2023-10-30', '2023-11-02', '2023-11-12', '2023-11-25', '2023-11-26', '2023-11-27', '2023-11-29', '2023-11-30', '2023-12-01', '2023-12-02', '2023-12-03', '2023-12-04', '2023-12-05', '2023-12-06', '2023-12-07', '2023-12-08', '2023-12-09', '2023-12-10', '2023-12-11', '2023-12-12', '2023-12-13', '2023-12-14', '2023-12-15', '2023-12-16', '2023-12-17', '2023-12-18', '2023-12-19', '2023-12-20', '2023-12-21', '2023-12-22', '2023-12-23', '2023-12-24', '2023-12-25', '2023-12-26', '2023-12-27', '2023-12-28', '2023-12-31', '2024-01-01', '2024-01-02', '2024-01-07', '2024-01-10', '2024-01-11', '2024-01-12', '2024-01-13', '2024-01-14', '2024-01-15', '2024-01-17', '2024-01-18', '2024-01-20', '2024-01-21', '2024-01-22', '2024-01-24', '2024-01-25', '2024-01-26', '2024-01-27', '2024-01-28', '2024-01-29', '2024-01-31', '2024-02-01', '2024-02-02', '2024-02-03', '2024-02-04', '2024-02-05', '2024-02-07', '2024-02-08', '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12', '2024

In [26]:
# retourne les dernières dates de snapshot pour chaque année parmi les dates complètes
def get_last_snapshot_dates_per_year(full_snapshot_dates):
    last_snapshot_dates = []
    for year in unique_years:
        yearly_dates = [date for date in full_snapshot_dates if date.startswith(year)]
        if yearly_dates:
            last_date = max(yearly_dates)
            last_snapshot_dates.append(last_date)
    return last_snapshot_dates

last_snapshot_dates = get_last_snapshot_dates_per_year(full_snapshot_dates)
print(sorted(last_snapshot_dates))

['2023-12-31', '2024-12-27', '2025-04-12']


In [27]:
# extraction des données complètes pour les années 2023 à 2025
top_50_2023_to_2025 = df[df['snapshot_date'].isin(last_snapshot_dates)]

In [28]:
top_50_2023_to_2025.head()

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,duration_ms,album_name,album_release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
187209,7so0lgd0zP2Sbgs2d7a1SZ,Die With A Smile,"Lady Gaga, Bruno Mars",1,0,0,NaN,2025-04-12,89,False,251667,MAYHEM,2025-03-07,0.519,0.601,6,-7.727,0,0.0317,0.2890,0.000000,0.126,0.498,157.964,3
187210,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,2,0,0,NaN,2025-04-12,99,False,210373,HIT ME HARD AND SOFT,2024-05-17,0.747,0.507,2,-10.171,1,0.0358,0.2000,0.060800,0.117,0.438,104.978,4
187211,2RkZ5LkEzeHGRsmDqKwmaJ,Ordinary,Alex Warren,3,0,0,NaN,2025-04-12,85,False,186964,"You'll Be Alright, Kid (Chapter 1)",2024-09-26,0.368,0.694,2,-6.141,1,0.0600,0.7040,0.000007,0.055,0.391,168.115,3
187212,4wJ5Qq0jBN4ajy7ouZIV1c,APT.,"ROSÉ, Bruno Mars",4,0,0,NaN,2025-04-12,92,False,169917,rosie,2024-12-06,0.777,0.783,0,-4.477,0,0.2600,0.0283,0.000000,0.355,0.939,149.027,4
187213,2CGNAOSuO1MEFCbBRgUzjd,luther (with sza),"Kendrick Lamar, SZA",5,0,0,NaN,2025-04-12,94,False,177598,GNX,2024-11-21,0.707,0.575,2,-7.546,1,0.1250,0.2510,0.000000,0.248,0.576,138.008,4


In [29]:
# vérification des valeurs manquantes
top_50_2023_to_2025.isnull().sum()

spotify_id              0
name                    0
artists                 0
daily_rank              0
daily_movement          0
weekly_movement         0
country               150
snapshot_date           0
popularity              0
is_explicit             0
duration_ms             0
album_name              4
album_release_date      4
danceability            0
energy                  0
key                     0
loudness                0
mode                    0
speechiness             0
acousticness            0
instrumentalness        0
liveness                0
valence                 0
tempo                   0
time_signature          0
dtype: int64

## 3. Sauvegarde des résultats

In [30]:
# sauvegarde des résultats
path = '../../data/top_50_71Countries_from_2023_to_2025.csv'
os.makedirs('../../data', exist_ok=True)
top_50_2023_to_2025.to_csv(path, index=False)